In [1]:
import requests
import json
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import numpy as np

In [67]:
for i in range(1):
    print(i)

0


In [75]:
SEARCH_TERM='jordan 1'
PAGE_COUNT=1


#Get top shoe IDs
url = 'https://2fwotdvm2o-dsn.algolia.net/1/indexes/product_variants_v2/query?x-algolia-agent=Algolia for vanilla JavaScript 3.25.1&x-algolia-application-id=2FWOTDVM2O&x-algolia-api-key=ac96de6fef0e02bb95d433d8d5c7038a'

shoeList = pd.DataFrame(columns=['product_template_id', 'name', 'main_picture_url'])

for i in range(PAGE_COUNT):
    data = {'distinct':'true', 'hitsPerPage':'50','page':i, 'query': SEARCH_TERM}
    r = requests.post(url, data=json.dumps(data))
    l = pd.DataFrame.from_records(r.json()['hits'])
    l = l[['product_template_id', 'name', 'main_picture_url']]
    shoeList = shoeList.append(l)
shoeList.rename(columns={'product_template_id': 'id', 'main_picture_url': 'picture'}, inplace=True)
shoeList

,id,name,picture
0,567948,Air Jordan 1 Retro High OG 'Dark Mocha',https://image.goat.com/750/attachments/product...
1,482560,Air Jordan 1 Retro High OG 'Obsidian',https://image.goat.com/750/attachments/product...
2,519961,Air Jordan 1 Retro High OG 'Royal Toe',https://image.goat.com/750/attachments/product...
3,491212,Air Jordan 1 Retro High OG 'Gym Red',https://image.goat.com/750/attachments/product...
4,485842,Air Jordan 1 Retro High OG 'Crimson Tint',https://image.goat.com/750/attachments/product...
5,602213,Air Jordan 1 Retro High OG 'Volt Gold',https://image.goat.com/750/attachments/product...
6,517360,Air Jordan 1 Retro High OG 'Court Purple 2.0',https://image.goat.com/750/attachments/product...
7,618984,Air Jordan 1 Retro High co.JP 'Midnight Navy' ...,https://image.goat.com/750/attachments/product...
8,693245,Air Jordan 1 Mid 'Banned',https://image.goat.com/750/attachments/product...
9,576959,Air Jordan 1 Mid 'Chicago Black Toe',https://image.goat.com/750/attachments/product...


In [79]:
shoeDict = {}
for _, shoe in shoeList.iterrows():
    print('Processing', shoe['name'])
    #Get shoe info
    url = 'https://www.goat.com/web-api/v1/product_variants'
    param = {'productTemplateId':shoe['id']} 
    header = {'User-Agent': 'Mozilla/5.0'}
    req = requests.get(url, headers=header, params=param)
    jsonInfo = req.json()
    #Turn to dataframe and clean up
    s = pd.DataFrame.from_records(jsonInfo)
    s.drop(['sizeOption', 'lowestPriceCentsByWarehouse', 'instantShipLowestPriceCents'], axis=1, inplace=True)
    s.rename(columns={'lowestPriceCents': 'price'}, inplace=True)
    s['price'] = [re.findall('(\d+)00', str(x))[0] for x in s['price']]
    s['price'] = s['price'].astype(int)
    shoeDict[shoe['name']] = s

Processing Air Jordan 1 Retro High OG 'Dark Mocha'


ValueError: If using all scalar values, you must pass an index

In [62]:
#compare new_with_defect any box cond with new_no_defect any box cond
for name, df in shoeDict.items():
    p = df[(df.shoeCondition != 'used')]
    withDefectDf = p.loc[p['shoeCondition']=='new_with_defects']
    noDefectDf = p.loc[p['shoeCondition']=='new_no_defects']
    
    withDefectDf = withDefectDf.loc[withDefectDf['size'].isin(noDefectDf['size'])]
    noDefectDf = noDefectDf.loc[noDefectDf['size'].isin(withDefectDf['size'])]
    
    withDefectDf = withDefectDf.groupby('size').min()
    noDefectDf = noDefectDf.groupby('size').min()
    diff = {}
    for s in withDefectDf.index:
        diff[s] = noDefectDf.loc[s]['price'] - withDefectDf.loc[s]['price']
    for i in list(diff.keys()):
        if diff[i] <= 0:
            del diff[i]
    if len(diff) != 0:
        print(name, '\n', diff)

Air Jordan 1 Mid 'Día de Muertos' 
 {6.5: 25}
Luka Doncic x Air Jordan 1 Mid SE 'Pregame Pack - Mindfulness' 
 {9.5: 43, 10.5: 19, 11.0: 5}
Air Jordan 1 Mid SE 'Varsity Purple' 
 {8.0: 28, 11.5: 26}
Wmns Air Jordan 1 Low 'Triple White' 
 {6.0: 26}
Wmns Air Jordan 1 Mid 'White University Red' 
 {7.0: 42}
Air Jordan 1 Low 'Court Purple' 
 {9.5: 15}
Air Jordan 1 Retro High OG 'Couture' 
 {13.0: 15}
Air Jordan 1 Mid 'Noble Red' 
 {9.5: 29}
Air Jordan 1 Retro High Zoom 'Fearless' 
 {8.0: 31, 9.0: 28, 11.0: 27, 13.0: 54, 14.0: 5}
Air Jordan 1 Mid SE 'Olympic' 
 {10.5: 25, 13.0: 48}
Fragment Design x Air Jordan 1 Retro High OG 
 {8.5: 2732}
Air Jordan 1 Retro High OG GS 'Shadow' 2018 
 {4.0: 36, 6.0: 71, 6.5: 75, 7.0: 50}
Air Jordan 1 Mid SE 'Cyber Active Fuchsia' 
 {9.5: 20}
Air Jordan 1 Retro High OG BG 'Top 3' 
 {7.0: 393}
Maison Chateau Rouge x Air Jordan 1 Mid SE 'Fearless' 
 {9.5: 72, 12.0: 140}
Air Jordan 1 Retro High OG 'Metallic Red' 
 {8.5: 7, 11.0: 20}
Air Jordan 1 Retro High OG NR

In [42]:
for i in list(diff.keys()):
    if diff[i] <= 0:
        del diff[i]
diff

{8.5: 9, 10.5: 8, 11.5: 21, 13.0: 23}

In [64]:
len(shoeDict)

50

In [32]:
withDefectDf

,shoeCondition,boxCondition,price
size,,,
5.5,new_no_defects,good_condition,2000
7.0,new_no_defects,good_condition,1070
7.5,new_no_defects,good_condition,2000
8.0,new_no_defects,good_condition,515
8.5,new_no_defects,badly_damaged,520
9.0,new_no_defects,good_condition,545
9.5,new_no_defects,badly_damaged,460
10.0,new_no_defects,badly_damaged,480
10.5,new_no_defects,badly_damaged,415


In [31]:
noDefectDf

,shoeCondition,boxCondition,price
size,,,
5.5,new_no_defects,good_condition,2000
7.0,new_no_defects,good_condition,1070
7.5,new_no_defects,good_condition,2000
8.0,new_no_defects,good_condition,515
8.5,new_no_defects,badly_damaged,520
9.0,new_no_defects,good_condition,545
9.5,new_no_defects,badly_damaged,460
10.0,new_no_defects,badly_damaged,480
10.5,new_no_defects,badly_damaged,415


In [30]:
df

,size,shoeCondition,boxCondition,price
0,5.5,new_no_defects,good_condition,2000
1,7.0,new_no_defects,good_condition,1070
2,7.5,new_no_defects,good_condition,2000
3,8.0,new_no_defects,good_condition,515
4,8.5,new_no_defects,good_condition,529
5,8.5,used,no_original_box,285
6,8.5,used,good_condition,460
7,8.5,new_with_defects,good_condition,699
8,8.5,new_with_defects,badly_damaged,520
9,9.0,new_no_defects,good_condition,545


In [26]:
diff = {}

diff

8.5
520
10.0
550
10.5
415
11.5
439
13.0
440


{8.5: 0, 10.0: -70, 10.5: 0, 11.5: 0, 13.0: 0}

In [5]:
for name, df in shoeDict.items():
    print(name)
    p = df[(df.shoeCondition != 'new_no_defects')]
    display(p)

Air Jordan 1 Retro High OG 'Dark Mocha'


,size,shoeCondition,boxCondition,price
7,6.0,new_with_defects,good_condition,500
14,8.5,used,good_condition,450
16,9.0,used,no_original_box,440
21,9.5,new_with_defects,good_condition,470
22,9.5,used,good_condition,448
23,9.5,new_with_defects,no_original_box,627
24,10.0,used,good_condition,460
26,10.0,new_with_defects,good_condition,448
27,10.5,used,good_condition,410
30,10.5,new_with_defects,badly_damaged,2000


Air Jordan 1 Retro High OG 'Obsidian'


,size,shoeCondition,boxCondition,price
3,7.5,used,good_condition,550
4,7.5,new_with_defects,good_condition,605
8,8.0,used,good_condition,490
9,8.0,used,badly_damaged,486
10,8.0,new_with_defects,good_condition,488
13,8.5,used,no_original_box,425
14,8.5,new_with_defects,good_condition,525
15,8.5,used,good_condition,400
17,9.0,new_with_defects,good_condition,495
18,9.0,used,no_original_box,477


Air Jordan 1 Retro High OG 'Royal Toe'


...

,size,shoeCondition,boxCondition,price
1,7.0,used,good_condition,400
4,8.5,used,good_condition,260
6,8.5,used,badly_damaged,315
7,9.0,used,no_original_box,240
9,9.0,used,good_condition,275
10,9.0,new_with_defects,good_condition,338
12,9.5,used,badly_damaged,230
14,9.5,used,good_condition,230
15,9.5,used,no_original_box,170
16,10.0,used,good_condition,210


Air Jordan 1 Low 'Reverse Bred'


,size,shoeCondition,boxCondition,price
4,7.0,new_with_defects,good_condition,140
9,8.0,new_with_defects,good_condition,114
10,8.0,used,no_original_box,100
12,8.0,used,good_condition,100
15,8.5,new_with_defects,good_condition,112
16,9.0,new_with_defects,good_condition,120
18,9.0,used,no_original_box,105
19,9.0,used,good_condition,110
23,9.5,used,good_condition,95
26,10.0,used,good_condition,88


Air Jordan 1 Retro High OG 'Pine Green'


,size,shoeCondition,boxCondition,price
5,8.5,used,no_original_box,285
6,8.5,used,good_condition,460
7,8.5,new_with_defects,good_condition,699
8,8.5,new_with_defects,badly_damaged,520
10,9.0,used,good_condition,485
12,9.5,used,good_condition,380
16,10.0,used,good_condition,370
17,10.0,used,no_original_box,280
18,10.0,used,badly_damaged,420
19,10.0,new_with_defects,badly_damaged,550
